In [8]:
path = "/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data/scraped-data/20220621-052943/"
goal = "/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data/CRAFT-results/20220621-052943"

In [9]:
a = path.split("/")
a

['',
 'projectnb',
 'sparkgrp',
 'ml-herbarium-grp',
 'ml-herbarium-data',
 'scraped-data',
 '20220621-052943',
 '']

In [11]:
"/".join(path.split("/")[:-3]) + "/CRAFT-results/" + path.split("/")[-2]

'/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data/CRAFT-results/20220621-052943'

In [ ]:
def run_handwritten_recog(line_images, handwriting_line_recognition_net):
	def operation(line_image):
		line_image_size = (60, 800)
		line_image = handwriting_recognition_transform(line_image, line_image_size)
		line_character_prob = handwriting_line_recognition_net(line_image.as_in_context(ctx))
		return line_character_prob
	form_character_prob = []
	form_character_prob.append(map(operation(line_images)))
	return form_character_prob

def handwriting_recog(lines, num_threads):
	print(lines)
	handwriting_line_recognition_net = HandwritingRecognitionNet(rnn_hidden_states=512,
																rnn_layers=2, ctx=ctx, max_seq_len=160)
	pretrained_params = "models/herb_line_trained_on_all.params"
	handwriting_line_recognition_net.load_parameters(pretrained_params, ctx=ctx) # "models/handwriting_line8.params"
	handwriting_line_recognition_net.hybridize()
	
	print("Starting multiprocessing...")
	character_probs = []
	pool = mp.Pool(min(num_threads, len(lines)))
	# func = partial(run_handwritten_recog, line_images=inputs)
	for output in tqdm(pool.imap(run_handwritten_recog, lines), total=len(lines)):
		character_probs.append(output)
	pool.close()
	pool.join()
	print("Done.\n")
	return character_probs